In [1]:
import netCDF4 as nc
import numpy as np
from netCDF4 import Dataset
import dateutil.parser  #引入这个库来将日期字符串转成统一的datatime时间格式

# 记录城市对应的lon和lat维的索引i j                
ijDict={'深圳':[],'上海':[],'广州':[],'北京':[],'成都':[],
          '杭州':[],'重庆':[],'西安':[],'苏州':[],'武汉':[],
          '南京':[],'天津':[],'郑州':[],'长沙':[],'东莞':[],
          '佛山':[],'宁波':[],'青岛':[],'沈阳':[],'合肥':[]}
# ijDict={'郑州':[],'长沙':[],'东莞':[],'佛山':[],'宁波':[],'青岛':[],'沈阳':[],'合肥':[]}
with open("/Users/rumeng/Downloads/qingzang/test/ij2city.txt", "r") as f:
    for line in f.readlines():
        line = line.strip('\n').split()
        if line[2] in ijDict.keys():
            ijDict[line[2]].append([int(line[0]),int(line[1])])

In [2]:
# 获得每月的天数
def getDays(year):
    if year%4!=0:
        return 365
    elif year%100==0 and year%400!=0:
        return 365
    else:
        return 366

In [3]:
import json

#/Users/rumeng/Downloads/qingzang/forcing_01dy/lrad
path='/Users/rumeng/Downloads/qingzang/forcing_01dy'
labelpath='/Users/rumeng/Downloads/qingzang/forcing_01dy/'

# params=['wind']
##wind风速
##lrad地面向下长波辐射
##prec降水率 Precipitation rate
##pres近地表气压 Near surface air pressure
##shum近地表空气比湿度 Near surface air specific humidity
##srad地面向下短波辐射 Surface downward shortwave radiation
params=['wind','lrad','prec','pres','shum','srad']

data=[]
cityDate={}
objPath='/Users/rumeng/Downloads/qingzang/result/'
# yearInfo=['2014','2015','2016','2017','2018']
yearInfo=['2000','2001','2002','2003','2004','2005','2006',
          '2007','2008','2009','2010','2011','2012','2013',
          '2014','2015','2016','2017','2018']
# monthInfo=['07','08','09','10','11','12']
monthInfo=['01','02','03','04','05','06','07','08','09','10','11','12']
fileName='_CMFD_V0106_B-01_01dy_010deg_'
# /Users/rumeng/Downloads/qingzang/forcing_01dy/wind/wind_CMFD_V0106_B-01_01dy_010deg_200001-200012.nc

for city in ijDict.keys():
    list1=[] # 存储信息
    with open(objPath+city+'Temp.json', 'r',encoding='utf-8-sig') as f:
        lines = f.readlines()
        for line in lines:
            climateInfo=json.loads(line)
            list1.append(climateInfo)
    passdays=0
    for y in yearInfo:
        numOfGuance=len(ijDict[city])
        sumOfGuance=0
        
        thisday=passdays
        
        for paramName in params:
            print(city,y,paramName)
            ncTemp=Dataset(labelpath+paramName+'/'+paramName+fileName+y+'01-'+y+'12.nc')
            days=len(ncTemp.variables['time'])
            all_times = ncTemp.variables['time']
            sdt = dateutil.parser.parse(y+'-1-1T10:30:00')#"2000-1-1T00:00:00"
            edt = dateutil.parser.parse(y+'-12-31T10:30:00')#"2000-1-31T21:00:00"
            st_idx = nc.date2index(sdt, all_times)
            et_idx = nc.date2index(edt, all_times)
            time = nc.num2date(ncTemp.variables['time'][:],'hours since 1900-1-1 00:00:00').data
            
            paramInfo = ncTemp.variables[paramName][st_idx:et_idx+1,:,:].data
            thisday=passdays
            for j in range(days):
                numOfGuance=len(ijDict[city])
                sumOfGuance=0
                
                year=time[j].year
                month=time[j].month
                day=time[j].day  
                
                for i in ijDict[city]: 
                    if paramInfo[j][i[0]][i[1]]==-32767:
                        numOfGuance-=1
                        continue
                    else:
                        if paramInfo[j][i[0]][i[1]]<0:
                            print('paramInfo',paramInfo[j][i[0]][i[1]])
                        sumOfGuance+=paramInfo[j][i[0]][i[1]]
                if str(year)+'-'+str(month)+'-'+str(day)!=list1[thisday]["date"]:
                    print('日期不匹配')
                if numOfGuance!=0:
                    avgParam=round(sumOfGuance/numOfGuance,2)
#                     if avgParam>8 or avgParam<=0:
#                         print('avgParam:',avgParam)
                    list1[thisday][paramName] = str(avgParam)
#                     print(sumOfGuance,numOfGuance)
                else:
                    print('numOfGuance为0',city,y,j)
                    list1[thisday][paramName] = '0.00'
                thisday+=1
        passdays+=days
        
    length=len(list1)
    for k in range(length):
        j = json.dumps(list1[k],ensure_ascii=False)  
        with open(objPath+'test1/'+city+'.json', 'a',encoding='utf-8-sig') as json_file:
            json_file.write(j)
            json_file.write('\n') 
                

郑州 2000 wind
郑州 2000 lrad
郑州 2000 prec
郑州 2000 pres
郑州 2000 shum
郑州 2000 srad
郑州 2001 wind
郑州 2001 lrad
郑州 2001 prec
郑州 2001 pres
郑州 2001 shum
郑州 2001 srad
郑州 2002 wind
郑州 2002 lrad
郑州 2002 prec
郑州 2002 pres
郑州 2002 shum
郑州 2002 srad
郑州 2003 wind
郑州 2003 lrad
郑州 2003 prec
郑州 2003 pres
郑州 2003 shum
郑州 2003 srad
郑州 2004 wind
郑州 2004 lrad
郑州 2004 prec
郑州 2004 pres
郑州 2004 shum
郑州 2004 srad
郑州 2005 wind
郑州 2005 lrad
郑州 2005 prec
郑州 2005 pres
郑州 2005 shum
郑州 2005 srad
郑州 2006 wind
郑州 2006 lrad
郑州 2006 prec
郑州 2006 pres
郑州 2006 shum
郑州 2006 srad
郑州 2007 wind
郑州 2007 lrad
郑州 2007 prec
郑州 2007 pres
郑州 2007 shum
郑州 2007 srad
郑州 2008 wind
郑州 2008 lrad
郑州 2008 prec
郑州 2008 pres
郑州 2008 shum
郑州 2008 srad
郑州 2009 wind
郑州 2009 lrad
郑州 2009 prec
郑州 2009 pres
郑州 2009 shum
郑州 2009 srad
郑州 2010 wind
郑州 2010 lrad
郑州 2010 prec
郑州 2010 pres
郑州 2010 shum
郑州 2010 srad
郑州 2011 wind
郑州 2011 lrad
郑州 2011 prec
郑州 2011 pres
郑州 2011 shum
郑州 2011 srad
郑州 2012 wind
郑州 2012 lrad
郑州 2012 prec
郑州 2012 pres
郑州 2012 shum

青岛 2010 lrad
青岛 2010 prec
青岛 2010 pres
青岛 2010 shum
青岛 2010 srad
青岛 2011 wind
青岛 2011 lrad
青岛 2011 prec
青岛 2011 pres
青岛 2011 shum
青岛 2011 srad
青岛 2012 wind
青岛 2012 lrad
青岛 2012 prec
青岛 2012 pres
青岛 2012 shum
青岛 2012 srad
青岛 2013 wind
青岛 2013 lrad
青岛 2013 prec
青岛 2013 pres
青岛 2013 shum
青岛 2013 srad
青岛 2014 wind
青岛 2014 lrad
青岛 2014 prec
青岛 2014 pres
青岛 2014 shum
青岛 2014 srad
青岛 2015 wind
青岛 2015 lrad
青岛 2015 prec
青岛 2015 pres
青岛 2015 shum
青岛 2015 srad
青岛 2016 wind
青岛 2016 lrad
青岛 2016 prec
青岛 2016 pres
青岛 2016 shum
青岛 2016 srad
青岛 2017 wind
青岛 2017 lrad
青岛 2017 prec
青岛 2017 pres
青岛 2017 shum
青岛 2017 srad
青岛 2018 wind
青岛 2018 lrad
青岛 2018 prec
青岛 2018 pres
青岛 2018 shum
青岛 2018 srad
沈阳 2000 wind
沈阳 2000 lrad
沈阳 2000 prec
沈阳 2000 pres
沈阳 2000 shum
沈阳 2000 srad
沈阳 2001 wind
沈阳 2001 lrad
沈阳 2001 prec
沈阳 2001 pres
沈阳 2001 shum
沈阳 2001 srad
沈阳 2002 wind
沈阳 2002 lrad
沈阳 2002 prec
沈阳 2002 pres
沈阳 2002 shum
沈阳 2002 srad
沈阳 2003 wind
沈阳 2003 lrad
沈阳 2003 prec
沈阳 2003 pres
沈阳 2003 shum
沈阳 2003 srad